# Experiment 2: Impact of Operational Phenomena

> What is the financial impact of failure scenarios?
Derive cost based on SLA from Amazon AWS
Is my datacenter currently at risk?

### Replication
The experimental results are obtained by running the following command:
```bash
bin/radice run -r 4096 -p 80 portfolios/phenomena.yml -P portfolio=phenomena
```

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import radice

FIGURE_PATH = 'figures/02-phenomena'

os.makedirs(FIGURE_PATH, exist_ok=True)
radice.set()

In [ ]:
risk = pd.read_parquet('data/risk/portfolio=phenomena/scenario=failures', columns=['seed', 'timestamp', 'id', 'value', 'cost'], read_dictionary=['id'])
risk['group'] = risk['id'].str.split(':', n=1).str[0]
risk

In [ ]:
penalties = {
    1: [(0.85, 1.0), (0.875, 0.3), (0.90, 0.1)],
    2: [(0.95, 1.0), (0.975, 0.3), (0.99, 0.1)],
    2.5: [(0.95, 1.0), (0.99, 0.3), (0.995, 0.1)],
    3: [(0.99, 1.0), (0.995, 0.3), (0.999, 0.1)],
    4: [(0.999, 1.0), (0.9995, 0.3), (0.9999, 0.1)],
    5: [(0.9999, 1.0), (0.99995, 0.3), (0.99999, 0.1)],
    6: [(0.99999, 1.0), (0.999995, 0.3), (0.999999, 0.1)],
    7: [(0.999999, 1.0), (0.9999995, 0.3), (0.9999999, 0.1)],
}

In [ ]:
scenarios = pd.concat([
    radice.adjust_penalty(risk, penalties[1]).assign(target='0.9'),
    radice.adjust_penalty(risk, penalties[2]).assign(target='0.99'),
    radice.adjust_penalty(risk, penalties[2.5]).assign(target='0.995'),
    radice.adjust_penalty(risk, penalties[3]).assign(target='0.999'),
    radice.adjust_penalty(risk, penalties[4]).assign(target='0.9999'),
    radice.adjust_penalty(risk, penalties[5]).assign(target='0.99999'),
    radice.adjust_penalty(risk, penalties[6]).assign(target='0.999999'),
    radice.adjust_penalty(risk, penalties[7]).assign(target='0.9999999'),
])

In [ ]:
x = radice.compute_monthly_risk(scenarios, keys=['target', 'group'])
x.groupby(['target', 'group'])['cost'].describe()

In [ ]:
fig, ax = plt.subplots(figsize=radice.figsize(2.5))

order = [
    '0.9',
    '0.99',
    '0.995',
    '0.999',
    '0.9999',
    '0.99999',
    '0.999999',
    '0.9999999',
]

hue_order = ["customer", "company", "society"]

ax = sns.barplot(x='cost', y='target', hue='group', data=x, order=order, hue_order=hue_order, ax=ax, edgecolor='black')

ax.set_ylabel("Availability Target")
ax.set_yticklabels(['90\\%', '99\\%', '99.5\\%', '99.9\\%', '99.99\\%', '99.999\\%', '99.9999\\%', '99.99999\\%'])
ax.set_xlabel("Incurred Monthly Cost (\euro{})")

legend = ax.get_legend()
legend.set_title("Risk Class")
legend.get_texts()[0].set_text('Customer-facing risks')
legend.get_texts()[1].set_text('Company-facing risks')
legend.get_texts()[2].set_text('Society-facing risks')

In [ ]:
y = radice.compute_monthly_risk(scenarios[scenarios['group'] == 'customer'], keys=['target'])
y.groupby(['target'])['cost'].describe()

In [ ]:
fig, ax = plt.subplots(figsize=radice.figsize(2.2))

order = [
    '0.9',
    '0.99',
    '0.995',
    '0.999',
    '0.9999',
    '0.99999',
    '0.999999',
    '0.9999999',
]

ax = sns.boxplot(data=y.reset_index(), x='cost', y='target', ax=ax, order=order, showfliers=False, showmeans=True, palette=['C0'])

ax.set_ylabel("Availability Target")
ax.set_xlabel("Risk of SLA Penalties per Month (\euro{})")
ax.set_yticklabels(['90\\%', '99\\%', '99.5\\%', '99.9\\%', '99.99\\%', '99.999\\%', '99.9999\\%', '99.99999\\%'])

fig.savefig(os.path.join(FIGURE_PATH, 'sla-penalty.pgf'))

## CPU interference

In [ ]:
interference = pd.read_parquet('data/interference/portfolio=phenomena')
interference = interference.groupby(['scenario', 'seed']).sum().reset_index()
interference

In [ ]:
interference.groupby(['scenario']).describe()         

In [ ]:
fig, ax = plt.subplots(figsize=radice.figsize(1.5))

sns.boxplot(data=interference, x='interference', y='scenario', order=['none', 'no-failures', 'no-interference', 'baseline'], showfliers=False, showmeans=True, ax=ax, palette=['C0'])

ax.set_ylabel("Op. Phenomena")
ax.set_yticklabels(["None", "Interference", "Failures", "All"])
ax.set_xlabel("Contended CPU time [\si{\second}]");

fig.savefig(os.path.join(FIGURE_PATH, 'contended-cpu-time.pgf'))

In [ ]:
risk = pd.read_parquet('data/risk/portfolio=phenomena', columns=['seed', 'timestamp', 'id', 'cost', 'scenario'], read_dictionary=['id'])

In [ ]:
x = radice.compute_monthly_risk(risk, keys=['id', 'scenario'])

In [ ]:
x[x['id'] == 'customer:cpu_interference'].groupby('scenario')['cost'].describe()